In [3]:
import numpy as np
import pandas as pd
import requests
import config
import os
import time
import json
from pandas.io.json import json_normalize

##Schedule_a API guide: https://api.open.fec.gov/developers/#/receipts/get_schedules_schedule_a_

In [4]:
### MULTI-CANDIDATE SEARCH ###
#Set search for all 2020 (two-year transaction period) Maine Congressional candidates
fec_key = config.fec_key
cand_name = ['COLLINS, SUSAN','MCCONNELL, MITCH']
cycle='2020'

#Initialize ID
id=int(0)
parameters = {'election_year':cycle
            ,'q':cand_name[id]
            ,'api_key':fec_key}

#Initialize comm_ids
cand_list=[]
r_cands=[]

for x in range(2):
    
    parameters.update(q=cand_name[id])
    #Requests candidate info
    r_cands = requests.get('https://api.open.fec.gov/v1/candidates/search',params=parameters).json()
    
    #Locates and sets Committee ID from 'principal_committees' sub-array
    cand_list.append(json_normalize(data=r_cands['results'],record_path='principal_committees')[['committee_id','name','party']])
    id=id+1
    
#Print list to validate
cand_list

[  committee_id                 name party
 0    C00314575  COLLINS FOR SENATOR   REP,
   committee_id                        name party
 0    C00193342  MCCONNELL SENATE COMMITTEE   REP
 1    C00155051  MCCONNELL SENATE COMMITTEE   REP]

In [5]:
#C00155051 refers to old McConnell campaign committee
#Remove that item and reform list

#Collins & McConnell joined
comm_ids = [cand_list[0]['committee_id'],cand_list[1][0:1]['committee_id']]
comm_ids

[0    C00314575
 Name: committee_id, dtype: object, 0    C00193342
 Name: committee_id, dtype: object]

In [6]:
start = time.time()
## FOR LOOP TO COLLECT CONTRIBUTION RECORDS ##
#Initialize dataframe collector for itemized contribs
idfs=[]
udfs=[]
commid=0

#Initialize query dictionary
itemdict = {
    'per_page':'100'
    ,'sort':'contribution_receipt_date'
    ,'api_key':fec_key
    ,'two_year_transaction_period':cycle
    ,'last_index':[]
    ,'last_contribution_receipt_date':[]
    ,'committee_id':comm_ids[commid]
}

#Page through results for each committee id
for x in range(0,len(comm_ids)-1):

    #Get first itemized payload for a candidate
    r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=itemdict).json()
    
    #Last page variables
    while r['pagination']['last_indexes'] is not None:
        
        #Store results of payload
        idf = json_normalize(r['results'])
        idfs.append(idf)
                
        #Assign last_index and date values, update itemdict
        last_index=pd.to_numeric(r['pagination']['last_indexes']['last_index'])
        last_date=r['pagination']['last_indexes']['last_contribution_receipt_date']
        #Update dictionary with new indices
        itemdict.update([('last_index',last_index)
                        ,('last_contribution_receipt_date',last_date)])

        #Get next payload with updated dict
        r = requests.get('https://api.open.fec.gov/v1/schedules/schedule_a/',params=itemdict).json()
    
    commid+=1
    
    #Update dictionary with next candidate in list and reset last indices
    itemdict.update([('committee_id',comm_ids[commid])
                     ,('last_index',[])
                     ,('last_contribution_receipt_date',[])])

# Concatenate all dfs
end = time.time()
print('Job complete in: '+str(end-start)+' seconds')

Job complete in: 19.326602935791016 seconds


In [7]:
itemdf=pd.concat(idfs,sort=False,ignore_index=True)
itemdf=itemdf.drop_duplicates(subset='transaction_id')

In [8]:
len(itemdf.index)

4336

In [11]:
#Write itemized individual results to local CSV
cwd = os.getcwd()
itemdf.to_csv(cwd+'/data/collins-and-mitch-schedule-a.csv',index=None)

In [10]:
# #Write itemized individual raw results to Google Sheet
# cwd = os.getcwd()

# #Google Credentials
# gc = pygsheets.authorize(service_file=cwd+'/me-congress-2020-creds.json')

# #Select sheet and worksheet
# sh = gc.open('me-congress-2020')
# #sh = gc.open_by_key('1AKrgHT9NLpoddV16B7_M_0PEjJmMQAGtXJUnLCTDHjA')
# wks = sh[3]

# #Clear sheet before load
# wks.clear(start='A1',fields='*')

# #Write contribs dataframe to sheet
# wks.set_dataframe(itemdf,(1,1))